In [1]:
import os 

current directory

In [2]:
%pwd

'/Users/jihaad/Documents/2023_Learning/Project/Text_Summarizer_Project/research'

- we want make change that directory to the root directori like
- /Users/jihaad/Documents/2023_Learning/Project/Text_Summarizer_Project

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/jihaad/Documents/2023_Learning/Project/Text_Summarizer_Project'

entity

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path 
    source_URL:str 
    local_data_file:Path
    unzip_dir:Path 

configuration

In [6]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
        
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config

component

In [8]:
import os 
import urllib.request as request 
import zipfile
from src.textSummarizer.logging import logger
from src.textSummarizer.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config 
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename= self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info \n{headers}")
        else:
            logger.info(f"file already exists of size: {get_size(Path(self.config.local_data_file))}")
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extract the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

pipeline

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-09-19 17:37:16,929: INFO: common: yaml file: config/config.yaml loaded succesfully]
[2023-09-19 17:37:16,936: INFO: common: yaml file: params.yaml loaded succesfully]
[2023-09-19 17:37:16,940: INFO: common: created directory at: artifacts]
[2023-09-19 17:37:16,952: INFO: common: created directory at: artifacts/data_ingestion]
[2023-09-19 17:37:32,554: INFO: 3021991361: artifacts/data_ingestion/data.zip download! with following info 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: CFFC:1681:D0FD6:F8E9D:650977F6
Accept-Ranges: bytes
Date: Tue, 19 Sep 2023 10:29:11 GMT
Via: 1.1 varnish
X-Served-By: cache-qpg1222-QPG
X-Cache: MISS
X-Ca